## 1. Import Useful Module

In [ ]:
#%run LIB.py

### 2a. Import Sales file

In [ ]:
file = 'https://raw.githubusercontent.com/Stephenkkt/Python/main/sales.csv'
sales = pd.read_csv(file, parse_dates = ['date'], header = 0, index_col = 0, dtype = {'Amt':'float'})

sales['mth'] = sales['date'].to_numpy().astype('datetime64[M]')
print("Sales Table")
print(sales.head(5))

print("\nData Type")
print(sales.dtypes)

#Describe the dataset
print("\n")
print(sales.describe(include='all'))

print("\n")
print(sales.nunique(axis=0))

## 2b. Plot Simple Bar Chart - By Product

In [ ]:
pd.options.plotting.backend = "plotly"
sales.groupby(by=["Prod"]).sum()[["Amt"]].plot(kind="bar")

## 3. Import Product File

In [ ]:
prod = pd.read_csv('https://raw.githubusercontent.com/Stephenkkt/Python/main/Prod.csv', header = 0, index_col=0)

print("Product:\n")
print(prod.head(5))

print("\nData Type")
print(prod.dtypes)

## 4a. Import Branch File

In [ ]:
branch = pd.read_csv('https://raw.githubusercontent.com/Stephenkkt/Python/main/BR_F.csv', header = 0, index_col = 0)

print("BRANCH:\n")
print(branch.head(5))
print("\nData Type")
print(branch.dtypes)



## 4b. Clean branch address

In [ ]:
#Clean Postal Code
branch['postal'] = branch['Zip'].str[:3] + " " + branch['Zip'].str[-3:]
branch = branch.drop(columns = ['Zip'])

#Create new field 'full address'
branch['full_address'] = branch['Address'] + "," + " Ontario" + "," + " Canada"

print('\nBranch')
print(branch.head())

## 4c. Map branch coordination

In [ ]:
#find the coordination
geolocator = Nominatim(timeout=10, user_agent = "MyApp")
branch['gcode'] = branch.full_address.apply(geolocator.geocode)

branch['lat'] = [g.latitude for g in branch.gcode]
branch['long'] = [g.longitude for g in branch.gcode]
print("\nBranch with Coordinate")
print(branch.head())


## 5. Import Staff File

In [ ]:
staff = pd.read_csv('https://raw.githubusercontent.com/Stephenkkt/Python/main/staff.csv', header = 0, index_col = 0)

#staff = staff.reset_index()
print("STAFF:\n")
print(staff.head(5))

print("\nData Type")
print(staff.dtypes)

print("\n")
print(staff.nunique(axis=0))

## Import Target File

In [ ]:
Tar = pd.read_csv('https://raw.githubusercontent.com/Stephenkkt/Python/main/BR_TAR.csv', header = 0, index_col=0, dtype = {'Br_ID':'int64'})
Tar['Sales_Tar'] = Tar['Sales_Tar'].replace(',','',regex=True)
Tar['Sales_Tar'] = Tar['Sales_Tar'].astype('float')
Tar = Tar.reset_index(drop=False)
print(Tar.head())

## 6. Summary

* **a.Summary in Staff Level**

In [ ]:
#Summary in Staff Level
sum_sales_staff_1 = sales.groupby(['Staff_ID']).agg(Total=('Amt','sum'),
                                                    Count=('Amt','count'),
                                                    Avg_Tix=('Amt','mean')
                                                   )
print("\nSales by Staff")
print(sum_sales_staff_1)

* **Merge Staff Table for getting Branch ID**

In [ ]:
sum_sales_br_1 = pd.merge(sum_sales_staff_1, staff, how = 'inner', left_on = 'Staff_ID', right_on = 'Staff_ID')

print("\nSales by Staff with Branch")
print(sum_sales_br_1)

* **Summary in Branch Level**

In [ ]:
sum_sales_br_2 = sum_sales_br_1.groupby(['Br_ID']).agg(BR_Total = ('Total', 'sum'),
                                                    BR_Count = ('Count','sum')
                                                    )
sum_sales_br_2['BR_Avg_Tix']=sum_sales_br_2['BR_Total']/sum_sales_br_2['BR_Count']
print("\nSales by Branch")
print(sum_sales_br_2.head())
print("\nSales by Branch - Number of Record")
print(len(sum_sales_br_2))

* **Merge Branch Table for Brnach details and create rank**

In [ ]:
sum_sales_br_2 = pd.merge(sum_sales_br_2,branch, how = 'inner', left_on = 'Br_ID', right_on = 'Br_ID')

#Create Ranking
sum_sales_br_2['rank'] = sum_sales_br_2['BR_Total'].rank(ascending = False).astype(int)

print("\nSales by Branch")
print(sum_sales_br_2.head())

print("\nSales by Branch - Number of Record")
print(len(sum_sales_br_2))

## Summary by date

In [ ]:
sum_sales_date = sales.groupby('date').agg(Total = ('Amt', 'sum'),
                                           Count = ('Amt', 'count')
                                          )
sum_sales_date = sum_sales_date.reset_index(drop = False)
print("Sale by Date")
print(sum_sales_date)

## Summary by Category, Product and month

* **Summary by Product and month**

In [ ]:
sum_sales_prod_mth = sales.groupby(['mth','Prod']).agg(Total = ("Amt","sum"),
                                                       Count = ("Amt", "count")
                                                      )
sum_sales_prod_mth = sum_sales_prod_mth.reset_index(drop = False)
print("\nSale by Prod and mth")
print(sum_sales_prod_mth)


* **Merge Product Table**

In [ ]:
sum_sales_prod_mth = pd.merge(sum_sales_prod_mth,prod, how = 'inner', left_on = "Prod", right_on = "Prod")

print("\nSale by Prod and mth")
print(sum_sales_prod_mth)

## Branch sales in latest month

In [ ]:
curr_mth_sales = sales.copy()

curr_mth_sales2 = curr_mth_sales.loc[(sales['date'].max().to_numpy().astype('datetime64[M]') == sales['date'].to_numpy().astype('datetime64[M]'))]

print("min=" + str(curr_mth_sales2['date'].min()))

print("\nmax="+str(curr_mth_sales2['date'].max()))
sum_curr_mth_1 = curr_mth_sales2.groupby(['Staff_ID']).agg(Total = ('Amt', 'sum'),
                                                           Count = ('Amt', 'count')
                                                            )

sum_curr_mth_1 = pd.merge(sum_sales_staff_1, staff, how = 'inner', left_on = 'Staff_ID', right_on = 'Staff_ID')
sum_curr_mth_2 = sum_curr_mth_1.groupby(['Br_ID']).agg(Total = ('Total', 'sum'),
                                                           Count = ('Count', 'sum')
                                                            )

sum_curr_mth_2 = pd.merge(sum_curr_mth_2,Tar, how = 'inner', left_on='Br_ID', right_on='Br_ID')
sum_curr_mth_2 = pd.merge(sum_curr_mth_2,branch[['Br_ID','Name']], how = 'inner', left_on='Br_ID', right_on='Br_ID')

print(sum_curr_mth_2.head())

## Export Files

In [ ]:
sum_sales_prod_mth.to_csv('sum_prod_mth.csv', index = False, header = 1)
sum_sales_br_2.to_csv('sum_branch.csv', index = False, header = 1)
sum_sales_date.to_csv('sum_date.csv', index = False, header = 1)
sum_curr_mth_2.to_csv('sum_achievement.csv', index = False, header = 1)

## Show all generated dataframe

In [ ]:
%whos DataFrame

## 7. Create Map

In [ ]:
import folium

# Find the Latitude & Longtitude for the map
avg_lat=branch['lat'].mean()
avg_long=branch['long'].mean()

print(avg_lat)
print(avg_long)

map = folium.Map(
    height=700, width=700,
    titles='Branch Sales',
    location=[avg_lat, avg_long],
    zoom_start = 11
)
map


In [ ]:
# Map Colour 
def sales_rank(row):
    if row['rank'] <=5:
        return 'pink'
    elif row['rank'] >=46:
        return 'red'
    else:
        return 'blue'

def sales_icon(row):
    if row['rank'] <=5:
        return 'star'
    elif row['rank'] >=46:
        return 'exclamation'
    else:
        return 'fa-circle'

sum_sales_br_2['colors'] = sum_sales_br_2.apply(sales_rank, axis=1)
sum_sales_br_2['icons'] = sum_sales_br_2.apply(sales_icon, axis=1)
sum_sales_br_2['BR_Total_M']= "$" + (round(sum_sales_br_2['BR_Total']/100000,1)).astype(str) + "M"
print(sum_sales_br_2['icons'].unique())
print(sum_sales_br_2['colors'].unique())

In [ ]:
city = sum_sales_br_2.iloc[0]

folium.Marker(
    location=[city['lat'], city['long']],
).add_to(map)


for _, city in sum_sales_br_2.iterrows():
    folium.Marker(
        location = [city['lat'], city['long']],
        popup = [city['Name']],
        tooltip = [city['Name'],
                                      
                   "Sales: ",city['BR_Total_M'],
                   
                   'Rank: ',city['rank']],
        icon=folium.Icon(color=city['colors'], icon=city['icons'], prefix='fa')
    ).add_to(map)
map


### Export Map as html

In [ ]:
map.save('c:/users/user/python_trial/Branch_Sales.html')